In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
from pyDOE import lhs
import time
import time
import math


D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1ty

In [2]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
np.random.seed(1232)
tf.set_random_seed(1232)

In [3]:
class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x0, u0, tb, X_f, layers, lb, ub,u_lb,u_ub):
        
        #    lb = np.array([-1, 0])      ub = np.array([1, 1])
        
        X0 = np.concatenate((x0, 0*x0+0.0), 1)              #    初始     
        X_lb = np.concatenate((0*tb + lb[0], tb), 1)    #    边界-1
        X_ub = np.concatenate((0*tb + ub[0], tb), 1)    #    边界+1    
        
        self.lb = lb
        self.ub = ub
               
        self.x0 = X0[:,0:1]
        self.t0 = X0[:,1:2]

        self.x_lb = X_lb[:,0:1]
        self.t_lb = X_lb[:,1:2]
        self.hsadasjd=1

        self.x_ub = X_ub[:,0:1]
        self.t_ub = X_ub[:,1:2]
        
        self.x_f = X_f[:,0:1]
        self.t_f = X_f[:,1:2]
        self.u_lb=u_lb
        self.u_ub=u_ub
        #分别是初始时刻的实部和虚部
        self.u0 = u0
        self.losslossloss=[]
        # Initialize NNs
        self.layers = layers
        #返回初始的权重w和偏差b
        self.weights, self.biases = self.initialize_NN(layers)
        
        # tf Placeholders
        #形参 占位符，行数不确定，列数确定为1
        self.x0_tf = tf.placeholder(tf.float32, shape=[None, self.x0.shape[1]])
        self.t0_tf = tf.placeholder(tf.float32, shape=[None, self.t0.shape[1]])
        self.u_lb_tf = tf.placeholder(tf.float32, shape=[None, self.u_lb.shape[1]])
        self.u_ub_tf = tf.placeholder(tf.float32, shape=[None, self.u_ub.shape[1]])
        self.u0_tf = tf.placeholder(tf.float32, shape=[None, self.u0.shape[1]])
        self.x_lb_tf = tf.placeholder(tf.float32, shape=[None, self.x_lb.shape[1]])
        self.t_lb_tf = tf.placeholder(tf.float32, shape=[None, self.t_lb.shape[1]])
        self.x_ub_tf = tf.placeholder(tf.float32, shape=[None, self.x_ub.shape[1]])
        self.t_ub_tf = tf.placeholder(tf.float32, shape=[None, self.t_ub.shape[1]])
        self.x_f_tf = tf.placeholder(tf.float32, shape=[None, self.x_f.shape[1]])
        self.t_f_tf = tf.placeholder(tf.float32, shape=[None, self.t_f.shape[1]])

        # tf Graphs  进行预测
        self.u0_pred= self.net_uv(self.x0_tf, self.t0_tf)
        self.u_lb_pred= self.net_uv(self.x_lb_tf, self.t_lb_tf)
        self.u_ub_pred = self.net_uv(self.x_ub_tf, self.t_ub_tf)
        self.f_u_pred= self.net_f_uv(self.x_f_tf, self.t_f_tf)
        
        # Loss   8个损失函数相加
        self.loss3=tf.reduce_mean(tf.square(self.u0_tf - self.u0_pred))
        
        self.loss2=tf.reduce_mean(tf.square(self.f_u_pred))
                
        
        self.loss = tf.reduce_mean(tf.square(self.u0_tf - self.u0_pred))+ \
                    (tf.reduce_mean(tf.square(self.u_ub_tf  - self.u_ub_pred)) + \
                    tf.reduce_mean(tf.square(self.u_lb_tf  - self.u_lb_pred))) + \
                    tf.reduce_mean(tf.square(self.f_u_pred))   
 
        self.loss4 = tf.reduce_mean(tf.square(self.u_ub_tf  - self.u_ub_pred)) +\
                        tf.reduce_mean(tf.square(self.u_lb_tf  - self.u_lb_pred))

        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss, 
                                                                method = 'L-BFGS-B', 
                                                                options = {'maxiter': 100000,
                                                                           'maxfun': 100000,
                                                                           'maxcor': 50,
                                                                           'maxls': 50,
                                                                           'ftol' : 1.0 * np.finfo(float).eps})
        
        

        self.optimizer_Adam = tf.train.AdamOptimizer()
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)
                
        # tf session  配置Session运行参数&&GPU设备指定）
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                                     log_device_placement=True))
        #初始化模型的参数
        init = tf.global_variables_initializer()
        self.sess.run(init)
    def initialize_NN(self, layers):        
        weights = []
        biases = []
        num_layers = len(layers) 
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)        
        return weights, biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        #产生截断正态分布随机数，stddev是标准差，取值范围为[ 0 - 2 * stddev, 0+2 * stddev ]
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)
    
    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        
        #将初始输入X映射到-1到1之间为H
        H = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for l in range(0,num_layers-2):
            W = weights[l]
            b = biases[l]
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y
    
    def net_uv(self, x, t):
        X = tf.concat([x,t],1)
        
        uv = self.neural_net(X, self.weights, self.biases)


        return uv
    
    
    
    def net_f_uv(self, x, t):
        
        u = self.net_uv(x,t)       
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_xx = tf.gradients(u_x, x)[0]
        
        hhhhhh=0.15
        f_u = u_t-u_xx-2*tf.exp(1/((2*t-1)*(2*t-1)+hhhhhh))+tf.exp(1/((2*t-1)*(2*t-1)+hhhhhh))*(1-x*x)*4*(2*t-1)/((2*t-1)*(2*t-1)+hhhhhh)/((2*t-1)*(2*t-1)+hhhhhh)
        #f_u=u-tf.exp(1/((2*t-1)*(2*t-1)+0.5))*(1-x*x)
        #return f_u/1319.919299519142
        return f_u
    
    def callback(self, loss,f_u_pred,u0_pred,u_ub_pred,u_lb_pred):
        
        self.losslossloss.append(loss)
            #losslossloss2
        sss=self.hsadasjd
        if sss%20==0:
            losssss =tf.reduce_mean(tf.square(f_u_pred))
            array1 = losssss.eval(session=tf.Session())        
            tf_dict = {self.x0_tf: self.x0, self.t0_tf: self.t0,
                   self.u0_tf: self.u0,self.u_lb_tf:self.u_lb,self.u_ub_tf:self.u_ub,
                   self.x_lb_tf: self.x_lb, self.t_lb_tf: self.t_lb,
                   self.x_ub_tf: self.x_ub, self.t_ub_tf: self.t_ub,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}
        
            loss1123456=self.u0_tf
            lossskdajsdkas=self.sess.run(loss1123456, tf_dict)
            zkjxJXhz = tf.reduce_mean(tf.square(lossskdajsdkas - u0_pred))
            array2 = zkjxJXhz.eval(session=tf.Session())
        
            loss1123456=self.u_ub_tf
            lssss1=self.sess.run(loss1123456, tf_dict)
            loss112345sds6=self.u_lb_tf
            sadsk=self.sess.run(loss112345sds6, tf_dict)            
            
            zkjxJXhzs = tf.reduce_mean(tf.square( lssss1- u_ub_pred))+tf.reduce_mean(tf.square(sadsk  - u_lb_pred))
            array4 = zkjxJXhzs.eval(session=tf.Session())
            print('It: %d, Loss1: %.9e,loss2: %.9e Loss3: %.9e' % 
                      (sss,array2,array4,array1))
            
            log5=open("log5.txt",mode = 'a+', encoding = 'utf-8')
            print(array2,file=log5)
            log5.close() 
            log6=open("log6.txt",mode = 'a+', encoding = 'utf-8')
            print(array4,file=log6)
            log6.close()            
            log7=open("log7.txt",mode = 'a+', encoding = 'utf-8')
            print(array1,file=log7)
            log7.close()            
            
            

        sss=sss+1
        self.hsadasjd=sss    
    
    def train(self, nIter):   
        tf_dict = {self.x0_tf: self.x0, self.t0_tf: self.t0,
                   self.u0_tf: self.u0,self.u_lb_tf:self.u_lb,self.u_ub_tf:self.u_ub,
                   self.x_lb_tf: self.x_lb, self.t_lb_tf: self.t_lb,
                   self.x_ub_tf: self.x_ub, self.t_ub_tf: self.t_ub,
                   self.x_f_tf: self.x_f, self.t_f_tf: self.t_f}
        lossloss1 = []
        lossloss2 = []
        lossloss3=[]
        
        start_time = time.time()
        
        loss_value11 = self.sess.run(self.loss3, tf_dict)
        lossloss1.append(loss_value11)
        
        loss_value22 = self.sess.run(self.loss2, tf_dict)
        lossloss2.append(loss_value22)
        
        loss_value33 = self.sess.run(self.loss4, tf_dict)
        lossloss3.append(loss_value33)
        
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            # Print
            if it % 20 == 0:
                elapsed = time.time() - start_time
                
                
                loss_value11 = self.sess.run(self.loss3, tf_dict)
                lossloss1.append(loss_value11)
                
                loss_value22 = self.sess.run(self.loss2, tf_dict)
                lossloss2.append(loss_value22)
                
                loss_value33 = self.sess.run(self.loss4, tf_dict)
                lossloss3.append(loss_value33)
                
                print('It: %d, Loss1: %.9e,loss2: %.9e Loss3: %.9e,Time: %.2f' % 
                      (it, loss_value11,loss_value33,loss_value22, elapsed))
                start_time = time.time()
                log1=open("log1.txt",mode = 'a+', encoding = 'utf-8')
                print(loss_value11,file=log1)
                log1.close()
                log2=open("log2.txt",mode = 'a+', encoding = 'utf-8')
                print(loss_value33,file=log2)
                log2.close()
                log3=open("log3.txt",mode = 'a+', encoding = 'utf-8')
                print(loss_value22,file=log3)
                log3.close()
                
                

        self.optimizer.minimize(self.sess, 
                                feed_dict = tf_dict, 
                                fetches = [self.loss,self.f_u_pred,self.u0_pred,self.u_ub_pred,self.u_lb_pred], 
                                loss_callback = self.callback
                               )    
        
        
           
        
        return lossloss1,lossloss2
    
    def predict(self, X_star):
        
        tf_dict = {self.x0_tf: X_star[:,0:1], self.t0_tf: X_star[:,1:2]}
        
        u_star = self.sess.run(self.u0_pred, tf_dict)  
        
        
        tf_dict = {self.x_f_tf: X_star[:,0:1], self.t_f_tf: X_star[:,1:2]}
        
        f_u_star = self.sess.run(self.f_u_pred, tf_dict)
               
        return u_star,f_u_star
    def loss_show(self):
        return self.losslossloss

In [4]:
def heatsolution(x,t):
    return math.exp(1/((2*t-1)**2+0.15))*(1-x**2)


In [5]:
if __name__ == "__main__": 
         
    
    # Doman bounds
    lb = np.array([-1, 0])
    ub = np.array([1, 1])

    N0 = 1200                                      #初始点
    N_b = 1200                                     #边界点
    N_f = 10000                                #适配点
    layers = [2,50,50,50,50,1]  
    #读取真实解
    x=np.linspace(-1,1,1200).flatten()[:,None]   
    t=np.linspace(0,1,1200).flatten()[:,None]   
    res=np.zeros([len(x),len(t)])  
    for i in range(len(x)):
        for j in range(len(t)):
            res[i,j]=heatsolution(x[i],t[j])
    
    
    X, T = np.meshgrid(x, t)
    X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
    #选定初始点N0=700个点
    idx_x = np.random.choice(x.shape[0], N0, replace=False)   
    x0 = x[idx_x,:]
    u0 = res[idx_x,0:1]
    #选择N_b=700个边界点
    idx_t = np.random.choice(t.shape[0], N_b, replace=False)
    tb = t[idx_t,:]
    u_lb = res[0,idx_t]
    u_ub=res[-1,idx_t]
    #N_f=2500个随机搭配点   第一列位置 第二列时间
    X_f = lb + (ub-lb)*lhs(2, N_f)
    x0=np.array(x0).flatten()[:,None]
    u0=np.array(u0).flatten()[:,None]
    u_lb=np.array(u_lb).flatten()[:,None]
    u_ub=np.array(u_ub).flatten()[:,None]

In [6]:
model = PhysicsInformedNN(x0, u0,tb, X_f, layers, lb, ub,u_lb,u_ub)     

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [7]:
LOSS1,LOSS2=model.train(2000)

It: 0, Loss1: 3.295086861e+00,loss2: 2.629875951e-02 Loss3: 4.799967000e+06,Time: 0.74
It: 20, Loss1: 3.079829407e+01,loss2: 1.821010971e+01 Loss3: 4.792303000e+06,Time: 1.24
It: 40, Loss1: 1.191999207e+02,loss2: 1.326125336e+02 Loss3: 4.723885500e+06,Time: 1.22
It: 60, Loss1: 1.708109436e+02,loss2: 2.482551880e+02 Loss3: 4.571873000e+06,Time: 1.24
It: 80, Loss1: 2.168480225e+02,loss2: 3.388994141e+02 Loss3: 4.498799000e+06,Time: 1.23
It: 100, Loss1: 2.626766968e+02,loss2: 4.243741455e+02 Loss3: 4.460795500e+06,Time: 1.24
It: 120, Loss1: 3.074906311e+02,loss2: 5.075167847e+02 Loss3: 4.430674500e+06,Time: 1.24
It: 140, Loss1: 3.520095825e+02,loss2: 5.902617188e+02 Loss3: 4.403233000e+06,Time: 1.25
It: 160, Loss1: 3.969790039e+02,loss2: 6.738549805e+02 Loss3: 4.376269500e+06,Time: 1.26
It: 180, Loss1: 4.430112000e+02,loss2: 7.590728760e+02 Loss3: 4.348053500e+06,Time: 1.27
It: 200, Loss1: 4.907098999e+02,loss2: 8.461633301e+02 Loss3: 4.317061000e+06,Time: 1.24
It: 220, Loss1: 5.407543945

It: 1840, Loss1: 6.409511230e+03,loss2: 1.236877148e+04 Loss3: 2.850836750e+06,Time: 1.23
It: 1860, Loss1: 6.471392578e+03,loss2: 1.249372754e+04 Loss3: 2.852124750e+06,Time: 1.24
It: 1880, Loss1: 6.532913574e+03,loss2: 1.261740430e+04 Loss3: 2.848647750e+06,Time: 1.23
It: 1900, Loss1: 6.594104004e+03,loss2: 1.274051953e+04 Loss3: 2.837958250e+06,Time: 1.23
It: 1920, Loss1: 6.655008301e+03,loss2: 1.286354492e+04 Loss3: 2.826897500e+06,Time: 1.23
It: 1940, Loss1: 6.715653809e+03,loss2: 1.298644922e+04 Loss3: 2.824857500e+06,Time: 1.25
It: 1960, Loss1: 6.776071777e+03,loss2: 1.310800195e+04 Loss3: 2.826837500e+06,Time: 1.21
It: 1980, Loss1: 6.836213867e+03,loss2: 1.322895703e+04 Loss3: 2.813599750e+06,Time: 1.21
It: 20, Loss1: 6.893564453e+03,loss2: 1.334536133e+04 Loss3: 2.807488500e+06
It: 40, Loss1: 6.957991699e+03,loss2: 1.346978906e+04 Loss3: 2.804802000e+06
It: 60, Loss1: 7.199851562e+03,loss2: 1.392746973e+04 Loss3: 2.796399750e+06
It: 80, Loss1: 7.540666016e+03,loss2: 1.457476758

It: 1940, Loss1: 1.983742578e+04,loss2: 3.730604688e+04 Loss3: 2.491187500e+06
It: 1960, Loss1: 2.008802344e+04,loss2: 3.777179297e+04 Loss3: 2.486710750e+06
It: 1980, Loss1: 2.007398047e+04,loss2: 3.775286719e+04 Loss3: 2.482943250e+06
It: 2000, Loss1: 1.994944922e+04,loss2: 3.752222656e+04 Loss3: 2.479654750e+06
It: 2020, Loss1: 1.989314062e+04,loss2: 3.741837109e+04 Loss3: 2.477643000e+06
It: 2040, Loss1: 1.982367969e+04,loss2: 3.728787891e+04 Loss3: 2.474759250e+06
It: 2060, Loss1: 1.994419922e+04,loss2: 3.751600781e+04 Loss3: 2.472121500e+06
It: 2080, Loss1: 1.987478906e+04,loss2: 3.738991406e+04 Loss3: 2.470344000e+06
It: 2100, Loss1: 1.988528125e+04,loss2: 3.741473828e+04 Loss3: 2.468483500e+06
It: 2120, Loss1: 1.991777734e+04,loss2: 3.748529688e+04 Loss3: 2.466680750e+06
It: 2140, Loss1: 1.994584375e+04,loss2: 3.753878906e+04 Loss3: 2.464008250e+06
It: 2160, Loss1: 1.997902930e+04,loss2: 3.759911719e+04 Loss3: 2.462476750e+06
It: 2180, Loss1: 2.002774609e+04,loss2: 3.769649609e

It: 4020, Loss1: 2.702433984e+04,loss2: 5.060491406e+04 Loss3: 2.288791500e+06
It: 4040, Loss1: 2.712808203e+04,loss2: 5.078217969e+04 Loss3: 2.287608750e+06
It: 4060, Loss1: 2.707734766e+04,loss2: 5.069228906e+04 Loss3: 2.286415000e+06
It: 4080, Loss1: 2.710618359e+04,loss2: 5.074765625e+04 Loss3: 2.285175500e+06
It: 4100, Loss1: 2.710008984e+04,loss2: 5.074086719e+04 Loss3: 2.284295500e+06
It: 4120, Loss1: 2.719055859e+04,loss2: 5.090063672e+04 Loss3: 2.282458000e+06
It: 4140, Loss1: 2.725426758e+04,loss2: 5.100959375e+04 Loss3: 2.281028500e+06
It: 4160, Loss1: 2.729049414e+04,loss2: 5.107317188e+04 Loss3: 2.279638250e+06
It: 4180, Loss1: 2.728372070e+04,loss2: 5.106028906e+04 Loss3: 2.278976750e+06
It: 4200, Loss1: 2.734598047e+04,loss2: 5.117242969e+04 Loss3: 2.277362000e+06
It: 4220, Loss1: 2.738860938e+04,loss2: 5.124937500e+04 Loss3: 2.276394500e+06
It: 4240, Loss1: 2.744085938e+04,loss2: 5.134558594e+04 Loss3: 2.275457000e+06
It: 4260, Loss1: 2.749317383e+04,loss2: 5.142953906e

It: 6100, Loss1: 3.096593945e+04,loss2: 5.768114844e+04 Loss3: 2.162088250e+06
It: 6120, Loss1: 3.100542578e+04,loss2: 5.776957812e+04 Loss3: 2.160817500e+06
It: 6140, Loss1: 3.104924609e+04,loss2: 5.786473438e+04 Loss3: 2.159583750e+06
It: 6160, Loss1: 3.107375977e+04,loss2: 5.791211719e+04 Loss3: 2.158412500e+06
It: 6180, Loss1: 3.112624023e+04,loss2: 5.801363672e+04 Loss3: 2.157217500e+06
It: 6200, Loss1: 3.118646094e+04,loss2: 5.812286719e+04 Loss3: 2.155759500e+06
It: 6220, Loss1: 3.121771289e+04,loss2: 5.818676562e+04 Loss3: 2.154663000e+06
It: 6240, Loss1: 3.124837305e+04,loss2: 5.825526953e+04 Loss3: 2.153298000e+06
It: 6260, Loss1: 3.125199414e+04,loss2: 5.826222656e+04 Loss3: 2.152091250e+06
It: 6280, Loss1: 3.128960938e+04,loss2: 5.834689844e+04 Loss3: 2.150873000e+06
It: 6300, Loss1: 3.132623242e+04,loss2: 5.840917578e+04 Loss3: 2.149500500e+06
It: 6320, Loss1: 3.134424609e+04,loss2: 5.843664844e+04 Loss3: 2.148481750e+06
It: 6340, Loss1: 3.135266602e+04,loss2: 5.845583594e

...

It: 32920, Loss1: 2.051919922e+04,loss2: 3.218608594e+04 Loss3: 2.207367188e+05
It: 32940, Loss1: 2.053273242e+04,loss2: 3.220154688e+04 Loss3: 2.204867031e+05
It: 32960, Loss1: 2.059571680e+04,loss2: 3.225278125e+04 Loss3: 2.200909844e+05
It: 32980, Loss1: 2.069683594e+04,loss2: 3.233513477e+04 Loss3: 2.195837969e+05
It: 33000, Loss1: 2.072065625e+04,loss2: 3.235490234e+04 Loss3: 2.192150312e+05
It: 33020, Loss1: 2.064184180e+04,loss2: 3.232036914e+04 Loss3: 2.192753594e+05
It: 33040, Loss1: 2.061774414e+04,loss2: 3.231814062e+04 Loss3: 2.187613438e+05
It: 33060, Loss1: 2.052331836e+04,loss2: 3.224008594e+04 Loss3: 2.185678594e+05
It: 33080, Loss1: 2.053881250e+04,loss2: 3.222279102e+04 Loss3: 2.182652656e+05
It: 33100, Loss1: 2.065135742e+04,loss2: 3.226977148e+04 Loss3: 2.178239531e+05
It: 33120, Loss1: 2.059434961e+04,loss2: 3.224010938e+04 Loss3: 2.176949688e+05
It: 33140, Loss1: 2.054159961e+04,loss2: 3.218541797e+04 Loss3: 2.175042500e+05
It: 33160, Loss1: 2.053902930e+04,loss2:

It: 34980, Loss1: 2.103457227e+04,loss2: 3.160394922e+04 Loss3: 1.952935781e+05
It: 35000, Loss1: 2.104848633e+04,loss2: 3.161845703e+04 Loss3: 1.950639844e+05
It: 35020, Loss1: 2.103974414e+04,loss2: 3.162749609e+04 Loss3: 1.948820781e+05
It: 35040, Loss1: 2.111390430e+04,loss2: 3.167286914e+04 Loss3: 1.945370938e+05
It: 35060, Loss1: 2.113867383e+04,loss2: 3.168574219e+04 Loss3: 1.943362969e+05
It: 35080, Loss1: 2.113550391e+04,loss2: 3.166533203e+04 Loss3: 1.942235625e+05
It: 35100, Loss1: 2.109526758e+04,loss2: 3.162820117e+04 Loss3: 1.941332031e+05
It: 35120, Loss1: 2.105888672e+04,loss2: 3.158384570e+04 Loss3: 1.940198281e+05
It: 35140, Loss1: 2.107210938e+04,loss2: 3.157597266e+04 Loss3: 1.937982812e+05
It: 35160, Loss1: 2.116207617e+04,loss2: 3.160153516e+04 Loss3: 1.938546250e+05
It: 35180, Loss1: 2.117263672e+04,loss2: 3.160276953e+04 Loss3: 1.932497188e+05
It: 35200, Loss1: 2.113933008e+04,loss2: 3.155406641e+04 Loss3: 1.930979844e+05
It: 35220, Loss1: 2.115407617e+04,loss2:

It: 37040, Loss1: 1.963718164e+04,loss2: 2.993758008e+04 Loss3: 1.778964531e+05
It: 37060, Loss1: 1.958680273e+04,loss2: 2.994043750e+04 Loss3: 1.776139531e+05
It: 37080, Loss1: 1.955660938e+04,loss2: 2.990812500e+04 Loss3: 1.774382812e+05
It: 37100, Loss1: 1.953666797e+04,loss2: 2.990870508e+04 Loss3: 1.772300312e+05
It: 37120, Loss1: 1.959300586e+04,loss2: 2.996548828e+04 Loss3: 1.768526875e+05
It: 37140, Loss1: 1.965398633e+04,loss2: 2.996923438e+04 Loss3: 1.765054844e+05
It: 37160, Loss1: 1.970516406e+04,loss2: 3.004706641e+04 Loss3: 1.761654375e+05
It: 37180, Loss1: 1.969300977e+04,loss2: 3.006533203e+04 Loss3: 1.758666406e+05
It: 37200, Loss1: 1.971819336e+04,loss2: 3.009708203e+04 Loss3: 1.755280938e+05
It: 37220, Loss1: 1.973164062e+04,loss2: 3.010872266e+04 Loss3: 1.752312031e+05
It: 37240, Loss1: 1.970183594e+04,loss2: 3.011609375e+04 Loss3: 1.749750625e+05
It: 37260, Loss1: 1.965382227e+04,loss2: 3.008537500e+04 Loss3: 1.748488281e+05
It: 37280, Loss1: 1.961755273e+04,loss2:

It: 39100, Loss1: 2.008561719e+04,loss2: 2.947124609e+04 Loss3: 1.570341250e+05
It: 39120, Loss1: 2.009863086e+04,loss2: 2.946827930e+04 Loss3: 1.566859219e+05
It: 39140, Loss1: 2.014550000e+04,loss2: 2.950481055e+04 Loss3: 1.563173438e+05
It: 39160, Loss1: 2.012336328e+04,loss2: 2.949992969e+04 Loss3: 1.560405000e+05
It: 39180, Loss1: 2.010706641e+04,loss2: 2.951890820e+04 Loss3: 1.557772031e+05
It: 39200, Loss1: 2.012842383e+04,loss2: 2.956054297e+04 Loss3: 1.554249688e+05
It: 39220, Loss1: 2.006917773e+04,loss2: 2.956316992e+04 Loss3: 1.551286094e+05
It: 39240, Loss1: 2.002315430e+04,loss2: 2.957853906e+04 Loss3: 1.547221094e+05
It: 39260, Loss1: 1.997394336e+04,loss2: 2.957848242e+04 Loss3: 1.543663594e+05
It: 39280, Loss1: 2.001031055e+04,loss2: 2.966053906e+04 Loss3: 1.538161250e+05
It: 39300, Loss1: 2.005500000e+04,loss2: 2.971175781e+04 Loss3: 1.533138438e+05
It: 39320, Loss1: 2.005669531e+04,loss2: 2.973113672e+04 Loss3: 1.529566719e+05
It: 39340, Loss1: 2.006110742e+04,loss2:

It: 41160, Loss1: 1.922833203e+04,loss2: 2.837720312e+04 Loss3: 1.381253438e+05
It: 41180, Loss1: 1.922224414e+04,loss2: 2.835373438e+04 Loss3: 1.379395000e+05
It: 41200, Loss1: 1.924162891e+04,loss2: 2.835451758e+04 Loss3: 1.377255312e+05
It: 41220, Loss1: 1.924585352e+04,loss2: 2.835003516e+04 Loss3: 1.375143750e+05
It: 41240, Loss1: 1.925578906e+04,loss2: 2.835522266e+04 Loss3: 1.373175625e+05
It: 41260, Loss1: 1.927754688e+04,loss2: 2.836972852e+04 Loss3: 1.370970938e+05
It: 41280, Loss1: 1.927028906e+04,loss2: 2.837196484e+04 Loss3: 1.369191875e+05
It: 41300, Loss1: 1.927505078e+04,loss2: 2.839040430e+04 Loss3: 1.367400000e+05
It: 41320, Loss1: 1.928432617e+04,loss2: 2.842402344e+04 Loss3: 1.365440781e+05
It: 41340, Loss1: 1.930133789e+04,loss2: 2.844875781e+04 Loss3: 1.362986562e+05
It: 41360, Loss1: 1.929044727e+04,loss2: 2.844308203e+04 Loss3: 1.361761562e+05
It: 41380, Loss1: 1.932133594e+04,loss2: 2.847646680e+04 Loss3: 1.359602656e+05
It: 41400, Loss1: 1.933828125e+04,loss2:

In [8]:
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_pred, f_u_pred = model.predict(X_star)
u_star = res.T.flatten()[:,None]  
error_u1 = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_u2 = np.linalg.norm(u_star-u_pred,1)/len(u_star)
error_u3 = np.linalg.norm(u_star-u_pred,np.inf)
print('二范数Error u: %e' % (error_u1))
print('平均绝对Error u: %e' % (error_u2))
print('无穷范数Error u: %e' % (error_u3))


二范数Error u: 1.100708e+00
平均绝对Error u: 1.650593e+02
无穷范数Error u: 7.198726e+02
